In [40]:
import warnings
warnings.filterwarnings('ignore')

from pymongo import MongoClient
import pprint

import copy
import pandas as pd

# Requests sends and recieves HTTP requests.
import requests 
# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup as soup

from urllib.request import urlopen as uReq
from collections import OrderedDict

###############

In [75]:
my_url = 'https://www.makeupalley.com/product/searching?CategoryID=7&NumberOfReviews=100'

In [76]:
#opening connection, download the webpage
uClient = uReq(my_url)
#get html
page_html = uClient.read()
#close the request
uClient.close()

In [77]:
def make_soup(url):
    uClient = uReq(url)
    page_html = uClient.read()
    page_soup = soup(page_html, "html.parser")
    return page_soup

In [8]:
make_soup(my_url)

In [78]:
#html parser
page_soup = soup(page_html, "html.parser")

In [79]:
page_soup.h1

<h1><a href="/" routerlink="/"><img alt="MakeupAlley" class="mua-logo-pic" src="../../../img/logo/logo-mobile.svg"/></a></h1>

In [80]:
page_soup.p

<p class="results-info"> Displaying results 1-15 of 851 results found </p>

In [81]:
links = page_soup.findAll('a',{'class':'item-name'})

In [82]:
len(links)

30

In [83]:
#use OrderedDict to preserve the order
links = list(OrderedDict.fromkeys(links))
len(links)

15

In [19]:
links[0]['href']

'/product/showreview.asp/ItemId=206387/Detox-in-a-Jar-Exfoliating-Face-Mask/Tula/Masks'

In [85]:
#get 15 product links from each page
prod_url = []
for l in links:
    prod_url.append('https://www.makeupalley.com{}'.format(l['href']))

In [86]:
#get product name
uClient = uReq(prod_url[0])
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, 'html.parser')
product = page_soup.find('div',{'class':'headline'}).get_text()
product

'Moisturizing Skin CreamVanicream'

In [87]:
#brand name
uClient = uReq(prod_url[0])
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, 'html.parser')
brand = page_soup.find('span',{'class':'brand'}).get_text()
brand

'Vanicream'

In [88]:
#User names list
uClient = uReq(prod_url[0])
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, 'html.parser')
user = page_soup.find('div',{'class':'__UserLink__'}).get_text()
# users = []
# for i in range(len(user)):
#     users.append(user[i].get_text())
user

'Katwren'

In [314]:
#try reading reviews from first link from first page

uClient = uReq(prod_url[0])
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, 'html.parser')
review = page_soup.findAll('span',{'class': 'review-text'})
reviews = []
for i in range(len(review)):
    reviews.append(review[i].get_text())

    
print(len(reviews))


10


In [90]:
##need to figure out how to find out the total pages for each product review
# for i in range(1,5):
#     for j in range(1,5):
#         print(prod_url[i]+'?page={}#reviews'.format(j))

**Create some functions to iterate through all pages**

In [199]:
# #get url for every page 
# pages_url = []
# for i in range(1,1650):
#     pages_url.append('https://www.makeupalley.com/product/searching?Brand=0&BrandName=&CategoryID=7&q=&page={}'.format(i))

# pages_url[0:2]

In [249]:
#store pages link in text file
def write_page_url(page_count):
    for i in range(1,page_count+1):
        with open(file='pages_url.txt',mode='a',newline = '\n') as fd:
            fd.write('https://www.makeupalley.com/product/searching?CategoryID=7&NumberOfReviews=100&page={}\n'.format(i))

In [250]:
#write_page_url(57)

In [251]:
url = pd.read_csv('pages_url.txt', header=None)

In [252]:
#list page url
def list_page_url(page_count):
    lst=[]
    for i in range(1,page_count+1):
        lst.append('https://www.makeupalley.com/product/searching?CategoryID=7&NumberOfReviews=100&page={}'.format(i))
    return lst

In [253]:
pages_url =list_page_url(57)

In [298]:
len(pages_url)

57

In [254]:
def make_soup(url):
    uClient = uReq(url)
    page_html = uClient.read()
    page_soup = soup(page_html, "html.parser")
    return page_soup

In [308]:
prod_url = []
for l in pages_url:
    s = make_soup(l)
    prod_links = s.findAll('a',{'class':'item-name'})
    prod_links = list(OrderedDict.fromkeys(prod_links))
    for p in prod_links:
        prod_url.append('https://www.makeupalley.com{}'.format(p['href']))
        
print(prod_url)

['https://www.makeupalley.com/product/showreview.asp/ItemId=133879/Facial-Moisturizing-Lotion---AM-SPF-30/CeraVe/Moisturizers', 'https://www.makeupalley.com/product/showreview.asp/ItemId=86350/Moisturizing-Skin-Cream/Vanicream/Moisturizers', 'https://www.makeupalley.com/product/showreview.asp/ItemId=108880/Acne-Stress-Control-Triple-Action-Toner/Neutrogena/Toners', 'https://www.makeupalley.com/product/showreview.asp/ItemId=78411/Ultra-Facial-Cream/Kiehls/Moisturizers', 'https://www.makeupalley.com/product/showreview.asp/ItemId=31005/Positively-Radiant-Skin-Brightening-Daily-Scrub/Aveeno/Scrubs', 'https://www.makeupalley.com/product/showreview.asp/ItemId=38477/Positively-Radiant-Cleanser/Aveeno/Cleansers', 'https://www.makeupalley.com/product/showreview.asp/ItemId=193076/The-Ordinary-AHA-30-%2B-BHA-2-Peeling-Solution/Deciem/Masks', 'https://www.makeupalley.com/product/showreview.asp/ItemId=134709/Cure-Natural-Aqua-Gel/Unlisted-Brand/Treatments-(Face)', 'https://www.makeupalley.com/produ

In [309]:
len(prod_url)

851

In [299]:
# #get all product links from all pages &store in txt file - this will be 800 something
# def write_prod_url(pages_url_list):
#     for l in pages_url_list:
#         s = make_soup(l)
#         prod_links = s.findAll('a',{'class':'item-name'})
#         prod_links = list(OrderedDict.fromkeys(links))
#         #for _ in range(len(pages_url)):
#         for l in prod_links:
#             with open(file='prod_url.txt',mode='a',newline='\n') as fd:
#                 fd.write('https://www.makeupalley.com{}\n'.format(l['href']))

In [300]:
# write_prod_url(pages_url)

In [310]:
#get all pages from one review
def get_review_page_count(url):
    s = make_soup(url).find('a',{'class':'overall-rating'}).get_text()
    pages = ''
    for i in range(len(s)): 
        if (s[i].isdigit()): 
            pages = pages+ str(s[i])
    return math.ceil((int(pages))/10)

In [349]:
import math
all_links = []
for i in prod_url:
    pages = get_review_page_count(i)
    for j in range(1,pages):
        all_links.append(i+'?page={}#reviews'.format(j))

In [350]:
all_links

['https://www.makeupalley.com/product/showreview.asp/ItemId=133879/Facial-Moisturizing-Lotion---AM-SPF-30/CeraVe/Moisturizers?page=1#reviews',
 'https://www.makeupalley.com/product/showreview.asp/ItemId=133879/Facial-Moisturizing-Lotion---AM-SPF-30/CeraVe/Moisturizers?page=2#reviews',
 'https://www.makeupalley.com/product/showreview.asp/ItemId=133879/Facial-Moisturizing-Lotion---AM-SPF-30/CeraVe/Moisturizers?page=3#reviews',
 'https://www.makeupalley.com/product/showreview.asp/ItemId=133879/Facial-Moisturizing-Lotion---AM-SPF-30/CeraVe/Moisturizers?page=4#reviews',
 'https://www.makeupalley.com/product/showreview.asp/ItemId=133879/Facial-Moisturizing-Lotion---AM-SPF-30/CeraVe/Moisturizers?page=5#reviews',
 'https://www.makeupalley.com/product/showreview.asp/ItemId=133879/Facial-Moisturizing-Lotion---AM-SPF-30/CeraVe/Moisturizers?page=6#reviews',
 'https://www.makeupalley.com/product/showreview.asp/ItemId=133879/Facial-Moisturizing-Lotion---AM-SPF-30/CeraVe/Moisturizers?page=7#reviews',

In [351]:
len(all_links)

22745

In [360]:
s = make_soup(all_links[0])
brand = s.find('span',{'class':'brand'}).get_text()
brand
# lst = []
# for i in range(10):
#     lst.append(r[i].get_text())
# lst

'CeraVe'

In [361]:
def generate_df(all_links_list):

    review = []
    product = []
    brand = []
    user = []
    rating = []

    for indx, i in enumerate(all_links_list):
        #sleep(sleep_time)
        s = make_soup(i)
        review_c = s.findAll('span',{'class': 'review-text'})
        product_c = s.find('div',{'class':'headline'}).get_text()
        try:
            brand_c = s.find('span',{'class':'brand'}).get_text()
        except:
            AttributeError
            brand_c = ''
        user_c = s.findAll('div',{'class':'__UserLink__'})
        rating_c = s.findAll('span',{'class': 'rating-value'})
        print(indx)
        for j in range(10):
            review.append(review_c[j].get_text())
            product.append(product_c)
            brand.append(brand_c)
            user.append(user_c[j].get_text())
            rating.append(user_c[j].get_text())

    MA_df = pd.DataFrame(list(zip(product, brand, user, review, rating)), columns = ['Product','Brand','User','Review','Rating'])
    return MA_df

In [ ]:
generate_df(all_links)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [294]:
# def list_prod_url(pages_url_list):
#     prod_lst = []
#     for l in pages_url:
#         s = make_soup(l)
#         prod_links = s.findAll('a',{'class':'item-name'})
#         prod_links = list(OrderedDict.fromkeys(links))
#         for _ in range(len(pages_url)):
#             for l in prod_links:
#                 prod_lst.append('https://www.makeupalley.com{}\n'.format(l['href']))
#     return prod_lst

In [274]:
# def list_prod_url(page_count):
#     for l in pages_url:
#         s = make_soup(l)
#         prod_links = s.findAll('a',{'class':'item-name'})
#         prod_links = list(OrderedDict.fromkeys(links))
#         for _ in range(len(pages_url)):
#             for l in prod_links:
#     lst=[]
#     for i in range(1,page_count+1):
#         lst.append('https://www.makeupalley.com/product/searching?CategoryID=7&NumberOfReviews=100&page={}'.format(i))
#     return lst

'https://www.makeupalley.com/product/showreview.asp/ItemId=32897/Anti-Aging-Cleansing-Gel/Peter-Thomas-Roth/Cleansers'

In [235]:
# #in each product url, need to get all the reviews from all pages (10 reviews per page)
# for i in prod_url[0:2]:
#     s = make_soup(i)
#     review = page_soup.findAll('span',{'class': 'review-text'})


AttributeError: 'int' object has no attribute 'timeout'

In [ ]:
#create a row containing product name, brand name, user name, review text, rating to append to a dataframe

def create_row(page_url):
    #15 reviews per page
    for i in range(1,15): 
        uClient = uReq('https://www.makeupalley.com{}'.format(prod_url[2]))
        page_html = uClient.read()
        uClient.close()
    page_soup = soup(page_html, 'html.parser')
    product = page_soup.find('div',{'class':'headline'}).get_text()
    brand = page_soup.find('span',{'class':'brand'}).get_text()
    user = page_soup.find('div',{'class':'__UserLink__'}).get_text()
    review = page_soup.find('span',{'class': 'review-text'}).get_text()
    rating = page_soup.find('span',{'class': 'rating-value'}).get_text()
    row.extend(product, user, review, rating)

In [ ]:
# def clean_string(column):
#     return column.apply(lambda x: x.replace("\n",'',2)).apply(lambda x: x.replace('  ',''))

# def scrape_reviews(PATH, n_pages, sleep_time = 0.3):

#     product = []
#     brand = []
#     user = []
#     review = []
#     rating = []

#     for p in range(n_pages):

#         sleep(sleep_time)

#         http = requests.get(f'{PATH}{p}')
#         bsoup = BeautifulSoup(http.text, 'html.parser')
#         page_soup = soup(page_html, 'html.parser')
        
#         product_containers = page_soup.findAll('div',{'class':'headline'})
#         brand_containers = page_soup.findAll('span',{'class':'brand'})
#         user_containers = page_soup.findAll('div',{'class':'__UserLink__'})
#         review_containers = page_soup.findAll('span',{'class': 'review-text'})
#         rating_container = page_soup.findAll('span',{'class': 'rating-value'})
    

#         for x in range(len(data_containers)):    
#             review_c = review_containers[x]
#             headers.append(review_c.h2.a.text)
#             reviews.append(review_c.p.text)
#             reviewer = user_containers[x]
#             names.append(reviewer.h3.text)
#             rating = rating_container[x]
#             ratings.append(rating.div.attrs['class'][1][12])
            


#             prof = profile_link_containers[x]
#             link = 'https://www.trustpilot.com'+ prof.a['href']
#             c_profile = requests.get(f'{link}')
#             csoup = BeautifulSoup(c_profile.text, 'html.parser')
#             cust_container = csoup.find('div', class_ = 'user-summary-location')
#             locations.append(cust_container.text)
            
#     rev_df = pd.DataFrame(list(zip(product, brand, user, review, rating)),
#                   columns = ['Product','Brand','User','Review','Rating'])
    
#     rev_df.Product = clean_string(rev_df.Product)
#     rev_df.Brand = clean_string(rev_df.Brand)
#     rev_df.User = clean_string(rev_df.User)
#     rev_df.Review = clean_string(rev_df.Review)
#     rev_df.Rating = rev_df.Rating.astype('int')
    
#     return rev_df